In [9]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [10]:
import pandas as pd
import optuna
import torch
import torch.nn as nn
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining
from sklearn.model_selection import ParameterGrid
import json
from transformers import LongformerTokenizer, LongformerForSequenceClassification, AutoTokenizer
from utils.load_dataset import create_loaders
from utils.training_loop import training_loop
from utils.models import Baseline, SmallTransformer, TextCNN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [11]:
device = 'cuda:0'

train_ds = pd.read_csv('data/train_ds.csv', converters={'embs': lambda x: json.loads(x.replace("'", '"'))})
val_ds = pd.read_csv('data/val_ds.csv', converters={'embs': lambda x: json.loads(x.replace("'", '"'))})
test_ds = pd.read_csv('data/test_ds.csv', converters={'embs': lambda x: json.loads(x.replace("'", '"'))})

train_loader, val_loader, test_loader = create_loaders(train_ds, val_ds, test_ds, batch_size=64)

## 1. Эксперименты с моделями без тюнинга гиперпараметров  (3 балла)

- Усложним бейзлайн, заменив RNN на LSTM
- Попробуем CNN для текстов
- Попробуем маленькую transformer модельку
- TF-IDF + LogisticRegression

(Конечно, можно было бы взять большую предобученную модель, качество будет на порядки выше, но предположим, что наши бизнес-ограничения требуют очень дешевое обучение и инференс)

In [4]:
model_lstm = Baseline(embed_dim=64, hidden_size=64, output_size=7, use_rnn=False, num_layers=2).to(device)
training_loop(model_lstm, 'LSTM', device, train_loader, test_loader)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: goodevening13 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


100%|██████████| 20/20 [04:54<00:00, 14.74s/it]


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_f1,▁▂▅▁▆▃▆▆▆▇▇▇▇███▇▇██
test_recall_n,▅▃▅█▆▆▅▅▄▄▃▁▃▂▂▁▃▂▁▂
train_f1,▁▁▂▁▂▃▄▄▅▅▆▆▆▇▇▇▇▇██
train_loss,█▆▆▆▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁
train_recall_n,▆▅▆█▇▅▅▅▄▆▄▁▆▆▅▅▇▇▇▇
epoch,19
test_f1,0.76227
test_recall_n,0.86155
train_f1,0.92719
train_loss,0.25357


In [5]:
del model_lstm

In [6]:
model_cnn = TextCNN(embed_dim=64, output_size=7).to(device)
training_loop(model_cnn, 'CNN', device, train_loader, test_loader)

100%|██████████| 20/20 [04:44<00:00, 14.23s/it]


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_f1,▁▄▇▄▇▆█▆▆▄▆▇▃▇▅▆▆▄▅▅
test_recall_n,▆█▇▄▇▃█▄▆▂▂▅▁▄▂▁▃▃▁▁
train_f1,▁▂▂▃▃▄▄▅▅▅▆▆▆▇▇▇████
train_loss,█▇▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁
train_recall_n,▇▇▇▄▆▂█▃▆▁▄▇▃▇▆▅▇█▇▆
epoch,19
test_f1,0.72833
test_recall_n,0.81673
train_f1,0.93503
train_loss,0.23738


In [7]:
del model_cnn

In [8]:
model_t = SmallTransformer(embed_dim=64, num_heads=2, hidden_dim=64, output_size=7).to(device)
training_loop(model_t, 'SmallTransformer', device, train_loader, test_loader)

100%|██████████| 20/20 [05:26<00:00, 16.32s/it]


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_f1,▁▂▃▅▅▇▇▇▇██████▇█▇██
test_recall_n,▅▅▆█▇▇▇▇▄▅▄▄▄▃▆▃▅▁▃▃
train_f1,▁▁▂▂▂▃▃▄▅▅▆▆▆▇▇▇▇███
train_loss,█▇▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁
train_recall_n,▃▆▅█▇▅▇▇▁▂▃▄▂▃▆▄▆▁▄▆
epoch,19
test_f1,0.74093
test_recall_n,0.84064
train_f1,0.93329
train_loss,0.28997


In [9]:
del model_t

In [10]:
vectorizer = TfidfVectorizer(max_features=128256)
X_train = vectorizer.fit_transform(train_ds['text'])
model = LogisticRegression(max_iter=500)
model.fit(X_train, train_ds['enc_label'])

LogisticRegression(max_iter=500)

In [11]:
predictions = model.predict(X_train)
X_test = vectorizer.transform(test_ds['text'])
predictions_test = model.predict(X_test)

In [12]:
labels = train_ds['enc_label'].to_numpy()
labels_test = test_ds['enc_label'].to_numpy()

In [15]:
import wandb
from torcheval.metrics.functional import multiclass_f1_score
def calculate_recall_normal(predictions, labels):
    normal_idx = [i for i in range(len(labels)) if labels[i] == 7]
    recall_normal = (predictions[normal_idx] == 7).sum() / len(normal_idx)
    return recall_normal

f1_train = multiclass_f1_score(torch.Tensor(predictions).to(dtype=torch.int64),
                               torch.Tensor(labels).to(dtype=torch.int64), average='weighted', num_classes=7)

f1_test = multiclass_f1_score(torch.Tensor(predictions_test).to(dtype=torch.int64), 
                               torch.Tensor(labels_test).to(dtype=torch.int64), average='weighted', num_classes=7)
recall_n_train = calculate_recall_normal(predictions, labels)
recall_n_test = calculate_recall_normal(predictions_test, labels_test)

wandb.init(
        project="ml_sys_design",
        name='TF-IDF Classifier',
        group='model_comparison'
    )
for i in range(20):
    wandb.log({
            "epoch": i,
            "train_loss": 0,
            "train_recall_n": recall_n_train,
            "train_f1": f1_train,
            "test_recall_n": recall_n_test,
            "test_f1": f1_test,
        })
wandb.finish()

/var/tmp/ipykernel_963701/579590635.py:5: RuntimeWarning: invalid value encountered in scalar divide
  recall_normal = (predictions[normal_idx] == 7).sum() / len(normal_idx)


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
test_f1,0.75163
test_recall_n,nan
train_f1,0.73249
train_loss,0
train_recall_n,nan


In [16]:
used_tokenizer = AutoTokenizer.from_pretrained('unsloth/Meta-Llama-3.1-8B-Instruct')
model_name = "allenai/longformer-base-4096"
pretrained_tokenizer = LongformerTokenizer.from_pretrained(model_name)

def decode(ids):
    return used_tokenizer.decode(ids)

def tokenize_and_save_embs(text):
    emb = pretrained_tokenizer(text, max_length=1024, truncation=True, return_tensors="pt")
    emb.global_attention_mask = torch.zeros_like(emb.input_ids)
    emb.global_attention_mask[:, 0] = 1
    emb.global_attention_mask[emb.input_ids == pretrained_tokenizer.sep_token_id] = 1 
    return emb.input_ids, emb.global_attention_mask

train_ds['text'] = train_ds['embs'].apply(decode)
val_ds['text'] = val_ds['embs'].apply(decode)
test_ds['text'] = test_ds['embs'].apply(decode)

#train_ds['embs_new'] = train_ds['text'].apply(tokenize_and_save_embs)
#val_ds['embs_new'] = val_ds['text'].apply(tokenize_and_save_embs)
#test_ds['embs_new'] = test_ds['text'].apply(tokenize_and_save_embs)
#train_loader_pre, val_loader_pre, test_loader_pre = create_loaders(train_ds, val_ds, test_ds, batch_size=64, extra=True)

## 2. Эксперименты с моделями с тюнингом гиперпараметров  (4 балла)

берем две модели: LSTM и SmallTransformer

Сначала подбираем параметры для LSTM

In [5]:
def objective_optuna(trial):
    params = {
        "hidden_size": trial.suggest_categorical("hidden_dim", [32, 64, 128]),
        "num_layers": trial.suggest_int("num_layers", 1, 4),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5)
    }
    
    model = Baseline(embed_dim=params['hidden_size'], hidden_size=params['hidden_size'], output_size=7, use_rnn=False, 
                     num_layers=params['num_layers']).to(device)
    
    val_f1 = training_loop(model, 'LSTM', device, train_loader, val_loader, params=params)
    return val_f1

study = optuna.create_study(direction="maximize")
study.optimize(objective_optuna, n_trials=10)

[I 2025-03-25 20:43:26,520] A new study created in memory with name: no-name-200122c6-00cc-4c72-9ae2-89d0deb956e1
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: goodevening13 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


100%|██████████| 20/20 [06:18<00:00, 18.94s/it]


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_f1,▁▂▃▃▇███▇▇▇███▇▇▇▇▇▇
test_recall_n,▅▆█▇█▇▆▅▃▅▂▃▃▄▂▂▂▂▂▁
train_f1,▁▁▂▂▃▃▄▅▅▆▆▇▇▇▇▇▇███
train_loss,█▇▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁
train_recall_n,▃▅█▇█▇▆▁▁▅▁▂▆▆▅▆▇▆▇▅
epoch,19
test_f1,0.74277
test_recall_n,0.83528
train_f1,0.94039
train_loss,0.19723


[I 2025-03-25 20:49:49,829] Trial 0 finished with value: 0.7427650690078735 and parameters: {'hidden_dim': 128, 'num_layers': 3, 'dropout': 0.4464886137511739}. Best is trial 0 with value: 0.7427650690078735.


100%|██████████| 20/20 [05:14<00:00, 15.74s/it]


...

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_f1,▃▄▄▄▁▄▄▅▄▇▇▅▇▇▇█████
test_recall_n,▄▄▃▄█▆▇▆▄▆▅▃▄▃▄▄▄▃▂▁
train_f1,▁▂▂▂▁▃▃▃▃▄▅▅▅▆▆▆▇▇██
train_loss,█▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁▁
train_recall_n,▃▄▃▄█▆▇▆▄▆▅▄▃▁▅▅▅▃▄▂
epoch,19
test_f1,0.74166
test_recall_n,0.84211
train_f1,0.86327
train_loss,0.40918


[I 2025-03-25 21:34:23,506] Trial 8 finished with value: 0.7416589856147766 and parameters: {'hidden_dim': 32, 'num_layers': 2, 'dropout': 0.1915978973911459}. Best is trial 7 with value: 0.7526282668113708.


100%|██████████| 20/20 [06:43<00:00, 20.20s/it]


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_f1,▁▃▃▃▃▅▅▆▆██▇▇▇██▇███
test_recall_n,█▇▇▇▇▅▄▆▄▄▄▄▃▃▃▁▂▂▁▂
train_f1,▁▂▂▂▂▃▄▅▅▆▆▆▆▇▇▇████
train_loss,█▇▆▆▅▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁
train_recall_n,█▅▆▇█▆▁▆▁▄▅▄▇▇▇▄▇▅▃▆
epoch,19
test_f1,0.75078
test_recall_n,0.85283
train_f1,0.9316
train_loss,0.23706


[I 2025-03-25 21:41:10,286] Trial 9 finished with value: 0.7507827281951904 and parameters: {'hidden_dim': 128, 'num_layers': 4, 'dropout': 0.48174889407109966}. Best is trial 7 with value: 0.7526282668113708.


In [8]:
def train_pbt(config):
    model = Baseline(embed_dim=64, hidden_size=64, output_size=7, use_rnn=False, num_layers=1).to(device)
    
    val_f1 = training_loop(model, 'LSTM', device, train_loader, val_loader, lr=config["lr"], params=config)
    tune.report(float(val_f1.item()))

pbt_scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric="val_f1",
    mode="max",
    perturbation_interval=2,
    hyperparam_mutations={
        "lr": tune.loguniform(1e-5, 1e-3),
        "dropout": tune.uniform(0.1, 0.5),
    }
)

tune.run(
    train_pbt,
    config={
        "lr": 1e-4,
        "dropout": 0.3,
    },
    scheduler=pbt_scheduler,
    num_samples=5,
    resources_per_trial={
        "cpu": 2,
        "gpu": 0.5 if torch.cuda.is_available() else 0
    }
)

2025-03-25 23:01:49,045	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/alinashutova/anaconda3/envs/aquakv/lib/python3.11/site-packages/ray/tune/tune.py:730: UserWarning: Consider boosting PBT performance by enabling `reuse_actors` as well as implementing `reset_config` for Trainable.
  warnings.warn(
2025-03-25 23:01:49,123	WARNING callback.py:136 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


2025-03-25 23:01:49,130	WARNING tune_controller.py:2132 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (280 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent pending trials.
2025-03-25 23:01:49,133	WARNING tune_controller.py:2132 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (280 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent pending trials.
2025-03-25 23:04:07,160	ERROR tune_controller.py:1331 -- Trial task failed for trial train_pbt

Trial name
train_pbt_fca93_00000
train_pbt_fca93_00001


2025-03-25 23:04:11,497	ERROR tune_controller.py:1331 -- Trial task failed for trial train_pbt_fca93_00000
Traceback (most recent call last):
  File "/home/alinashutova/anaconda3/envs/aquakv/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/alinashutova/anaconda3/envs/aquakv/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/alinashutova/anaconda3/envs/aquakv/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/alinashutova/anaconda3/envs/aquakv/lib/python3.11/site-packages/ray/_private/worker.py", line 2782, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
                                  ^^^^^^

In [ ]:
import torch.optim as optim


grid_params = {
    "lr": [1e-3, 1e-4],
    "optimizer": [optim.Adam, optim.RMSprop],
}

best_val_f1 = 0
for params in ParameterGrid(grid_params):
    model = Baseline(embed_dim=64, hidden_size=64, output_size=7, use_rnn=False, 
                    num_layers=1).to(device)

    val_f1 = training_loop(model, 'LSTM', device, train_loader, val_loader, params=params, optimizer=params["optimizer"], lr=params['lr'])
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_params = params
    
    print("Лучшие параметры (Grid Search):", best_params)

100%|██████████| 20/20 [04:43<00:00, 14.18s/it]


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_f1,▁▂▃▂▅▆▇▇▇▇▇██▇▇▇▇▇▇▇
test_recall_n,▆▇▆█▇▅▇▅▄▄▅▄▄▃▂▂▂▄▂▁
train_f1,▁▁▂▂▂▃▄▅▅▆▆▆▆▇▇▇█▇██
train_loss,█▆▆▅▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁
train_recall_n,▅▆▅█▆▄▄▁▁▂▄▆▄▃▄▃▄▇▄▄
epoch,19
test_f1,0.75255
test_recall_n,0.87037
train_f1,0.94375
train_loss,0.21868


Лучшие параметры (Grid Search): {'lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>}


 85%|████████▌ | 17/20 [03:53<00:40, 13.66s/it]

То же самое для CNN

In [ ]:
def objective_optuna(trial):
    params = {
        "hidden_size": trial.suggest_categorical("embed_dim", [32, 64, 128]),
        "num_convs": trial.suggest_int("num_convs", 2, 5),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5)
    }
    
    model = TextCNN(embed_dim=params['embed_dim'], output_size=7, num_convs=params['num_convs'], dropout=params['dropout']).to(device)
    
    val_f1 = training_loop(model, 'CNN', device, train_loader, val_loader, params=params)
    return val_f1

study = optuna.create_study(direction="maximize")
study.optimize(objective_optuna, n_trials=6)

In [ ]:
def grid_search():
    grid_params = {
        "lr": [1e-3, 1e-4],
        "optimizer": [optim.Adam, optim.RMSprop],
    }
    
    best_val_f1 = 0
    for params in ParameterGrid(grid_params):
        model = Baseline(embed_dim=params['hidden_size'], hidden_size=params['hidden_size'], output_size=7, use_rnn=False, 
                     num_layers=params['num_layers']).to(device)
    
        val_f1 = training_loop(model, 'CNN', device, train_loader, val_loader, params=params, optimizer=params["optimizer"], lr=params['lr'])
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_params = params
    
    print("Лучшие параметры (Grid Search):", best_params)

## 4. Демо инференса модели  (2 балл)

В репозитории по ссылке https://github.com/goodevening13/last_hse

Все скриншоты также в репозитории

model = Baseline(embed_dim=64, hidden_size=64, output_size=7, use_rnn=False, num_layers=1, dropout=0.25).to(device)
    
_ = training_loop(model, 'LSTM', device, train_loader, val_loader, params=params)
model.push_to_hub("my-awesome-model")